<a href="https://colab.research.google.com/github/simnpeter/usernameApproving/blob/main/LoRA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q bitsandbytes datasets accelerate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 27.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# Select CUDA device index
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/flan-t5-large"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=BitsAndBytesConfig(load_in_8bit=True))
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [4]:
from google.colab import drive
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import DatasetDict, Dataset

drive.mount('/content/drive')

path='/content/drive/My Drive/Colab Notebooks/dataset.csv'

def custom_parser(line):
    parts = line.split(";")  # Felosztjuk a sort pontosvessző mentén
    # Az első elem az összes többi elem
    username = parts[0] if len(parts) == 1 else ";".join(parts[:-1])
    label = int(parts[-1])  # Utolsó elem a címke, ami számmá konvertálható
    return username, label

data = []
with open(path, "r") as file:
  for line in file:
        username, label = custom_parser(line.strip())
        data.append((username, label))

# Az adatok pandas DataFrame-be helyezése
df = pd.DataFrame(data, columns=["Username", "Label"])

df['Label'] = df['Label'].map({1: 'Approved', 0: 'Not approved'})

dataset_train, dataset_test = train_test_split(df, test_size=0.2, random_state=42)

dataset_train = Dataset.from_pandas(dataset_train)

dataset_test = Dataset.from_pandas(dataset_test)

dataset = DatasetDict({'train': dataset_train, 'test': dataset_test})

Mounted at /content/drive


In [5]:
from datasets import concatenate_datasets
import numpy as np

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset['train'], dataset['test']]).map(lambda x: tokenizer(x["Username"], truncation=True), batched=True, remove_columns=["Username", "Label"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 85))
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset['train'], dataset['test']]).map(lambda x: tokenizer(x["Label"], truncation=True), batched=True, remove_columns=["Username", "Label"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 90))
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/12726 [00:00<?, ? examples/s]

Max source length: 6


Map:   0%|          | 0/12726 [00:00<?, ? examples/s]

Max target length: 5


In [6]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["Is this username approvable: " + item for item in sample["Username"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["Label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["Username", "Label", "__index_level_0__"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

# save datasets to disk for later easy loading
tokenized_dataset["train"].save_to_disk("data/train")
tokenized_dataset["test"].save_to_disk("data/eval")

Map:   0%|          | 0/10180 [00:00<?, ? examples/s]

Map:   0%|          | 0/2546 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


Saving the dataset (0/1 shards):   0%|          | 0/10180 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2546 [00:00<?, ? examples/s]

In [7]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [8]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q", "v"], lora_dropout=0.05, bias="none", task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)

In [10]:
from transformers import TrainingArguments, Trainer

output_dir="/content/drive/My Drive/Colab Notebooks/lora-flan-t5-base"

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    learning_rate=1e-3,
    gradient_accumulation_steps=1,
    auto_find_batch_size=True,
    num_train_epochs=20,
    save_steps=100,
    save_total_limit=8,
    save_strategy="epoch",
    load_best_model_at_end =True
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,0.081100,0.071548
2,0.078600,0.072490


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 durin

In [ ]:
peft_model_id="/content/drive/My Drive/Colab Notebooks/results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Definiálj egy változót a mentett modell és tokenizer útvonalával
peft_model_id = "/content/drive/My Drive/Colab Notebooks/results"

# Töltsd be a modelled és a tokeneződet
model = T5ForConditionalGeneration.from_pretrained(peft_model_id)
tokenizer = T5Tokenizer.from_pretrained(peft_model_id)

In [ ]:
test = dataset['test']

df = test.to_pandas()

# Username oszlop módosítása
df['Username'] = ["Is this username approvable: " + username for username in df["Username"]]

# Módosított DataFrame visszaalakítása Dataset objektummá
test = Dataset.from_pandas(df)

In [ ]:
from datasets import load_dataset
import torch

# Betöltés és előkészítés a tesztkészlettel való predikcióhoz
# Például "Username" vagy bármi egyéb bemeneti oszlop neve
inputs = test['Username']

# Predikció végrehajtása a tesztkészleten
predictions = []
for input_text in inputs:
    # Tokenizáció és inputok előkészítése
    inputs = tokenizer(input_text, return_tensors="pt")
    # Predikció végrehajtása
    outputs = model.generate(**inputs)
    # Decodolás
    answer = tokenizer.decode(outputs[0],skip_special_tokens=True)
    # A predikció hozzáadása a listához
    predictions.append(answer)

# A predikciók hozzáadása a tesztkészlethez
#test["predicted_labels"] = predictions
predictions_df = pd.DataFrame({"predicted_labels": predictions})

# Concatenate the predictions dataframe with your test dataset
test_df = test.to_pandas()

# Concatenate the predictions DataFrame with the test DataFrame
test_with_predictions = pd.concat([test_df, predictions_df], axis=1)
# Eredmények kiértékelése vagy mentése


In [ ]:
from sklearn.metrics import accuracy_score

# Előrejelzések és címkék kinyerése
predicted_labels = test_with_predictions["predicted_labels"]
true_labels = test_with_predictions["Label"]  # Címkék oszlopnév

# Pontosság kiszámítása
accuracy = accuracy_score(true_labels, predicted_labels)

# Pontosság kiírása
print(f"A modell pontossága: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

class_names = ['Not approved', 'Approved']

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()